___

<a href='http://www.pieriandata.com'></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Cравнительный анализ различных методов регрессии!

## Данные

Вас только что наняла на работу буровая компания, которая применяет рентгеновские лучи для определения плотности камня. Задача в том, чтобы вовремя менять буровые головки прежде, чем переходить к бурению камня!

![image]<img src='boring.jpg' height=200>

Компания представила Вам результаты лабораторных тестов - в них измеряется отражённый сигнал, полученный сенсорами на разных частотах в герцах, для различной плотности камня. Оказалось, что если отобразить эти результаты на графике, то мы получим синусоиду - с изменением плотности камня уровень отражённого сигнала меняется волнообразно...

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("rock_density_xray.csv")

In [ ]:
df.head()

In [4]:
df.columns=['Signal',"Density"]

In [ ]:
plt.figure(figsize=(8,6),dpi=100)
sns.scatterplot(x='Signal',y='Density',data=df)

---
---

## Разбиение данных на обучающий и тестовый наборы 

Разбиение нужно для последующей оценки модели на тестовом наборе данных.

In [14]:
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature
# Если у нас только 1 признак, то у нас не дата фрейм, а обЬект сериес, следовательно, 
# его надо преобразовать в дата фрейм.
X = df['Signal'].values.reshape(-1,1)  
y = df['Density']

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

-----

# Линейная регрессия

In [20]:
from sklearn.linear_model import LinearRegression

In [21]:
lr_model = LinearRegression()

In [ ]:
lr_model.fit(X_train,y_train)

In [23]:
lr_preds = lr_model.predict(X_test)

In [ ]:
lr_preds

что-то не так...

In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
np.sqrt(mean_squared_error(y_test,lr_preds))

Как выглядит эта линия?

In [27]:
signal_range = np.arange(0,100)

In [28]:
lr_output = lr_model.predict(signal_range.reshape(-1,1))

In [ ]:
plt.figure(figsize=(8,6),dpi=70)
sns.scatterplot(x='Signal',y='Density',data=df,color='black')
plt.plot(signal_range,lr_output)

Видно, что это не нормально, но если признаков много, то визуализировать результат работы модели нельзя. Тогда следует посмотреть на остатки.

In [ ]:
# ______Анализ остатков __________________________
test_residuals = y_test - lr_preds
sns.scatterplot(x = y_test, y = test_residuals)
plt.axhline(y=0, color='red', ls='--')
sns.displot(test_residuals, bins=25, kde=True)
# plt.show()

Видно, что ошибки имеют закономерности, а должны быть случайными. Можно сравнить с нормальным распределением.

In [ ]:
# __________ Сравниваем с идеальным нормальным распределением
import scipy as sp 
# Создаём объект figure и оси для рисования графика
fig, ax = plt.subplots(figsize=(6,8),dpi=100)

# probplot возвращает значения, которые можно использовать при необходимости
# мы хотим просто нарисовать график, поэтому присваиваем эти значения переменной _
_ = sp.stats.probplot(test_residuals,plot=ax)

Увы, близко к нормальнму закону.Т.ч. - смотрим на первый график...

---

# Полиномиальная регрессия

### Вспомогательная функция для запуска моделей

In [50]:
def run_model(model,X_train,y_train,X_test,y_test):
    
    # Обучение модели
    model.fit(X_train,y_train)
    
    # Вычисление метрик
    
    preds = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test,preds))
    mae = mean_absolute_error(y_test, preds)
    print(f'MAE : {np.around(mae, 3)}')
    print(f'RMSE : {np.around(rmse, 3)}')
    
    # Рисуем график с результатами
    signal_range = np.arange(0,100)
    output = model.predict(signal_range.reshape(-1,1))
    
    
    plt.figure(figsize=(12,6),dpi=150)
    sns.scatterplot(x='Signal',y='Density',data=df,color='black')
    plt.plot(signal_range,output)

#### Lirear model (проверим работу ф-ции)

In [ ]:
run_model(lr_model,X_train,y_train,X_test,y_test)

## Polynomial model

#### Пайплайн для степеней полинома

In [44]:
from sklearn.pipeline import make_pipeline

In [45]:
from sklearn.preprocessing import PolynomialFeatures

Далее нам надо выполнить 2 шага - создание полиномиальных признаков (PolynomialFeatures), которые мы и подаем на вход линейной регрессии. 

In [ ]:
# Сравнение моделей с различными степенями полинома
collection = [2,3,4, 6, 8]
for i in collection:
    pipe = make_pipeline(PolynomialFeatures(degree=i),LinearRegression())
    print("___________")
    print(f'degrees = {i}')
    run_model(pipe,X_train,y_train,X_test,y_test)

______

## Регрессия KNN 

In [57]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
preds = {}
k_values = [1,5,10, 15]
for n in k_values:
    model = KNeighborsRegressor(n_neighbors=n)
    run_model(model,X_train,y_train,X_test,y_test)

---
## Деревья решений для регрессии

In [60]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dree_model = DecisionTreeRegressor()

run_model(dree_model,X_train,y_train,X_test,y_test)

In [62]:
dree_model.get_n_leaves()

270

----

## Метод опорных векторов для регрессии

In [63]:
from sklearn.svm import SVR

In [64]:
from sklearn.model_selection import GridSearchCV

In [65]:
param_grid = {'C':[0.01,0.1,1,5,10,100,1000],'gamma':['auto','scale']}
svr = SVR()

In [66]:
grid = GridSearchCV(svr,param_grid)

In [ ]:
run_model(grid,X_train,y_train,X_test,y_test)

In [ ]:
grid.best_estimator_

---

## Случайные леса для регрессии

In [69]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
trees = [10,50,100]
for n in trees:
    model = RandomForestRegressor(n_estimators=n)
    run_model(model,X_train,y_train,X_test,y_test)

## Градиентный бустинг

Мы изучим этот метод более подробно в следующем разделе.

In [71]:
from sklearn.ensemble import GradientBoostingRegressor

In [292]:
# help(GradientBoostingRegressor)

In [ ]:
   
gradient_model = GradientBoostingRegressor()

run_model(model,X_train,y_train,X_test,y_test)

## Adaboost

In [73]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
adaBoost_model = GradientBoostingRegressor()

run_model(model,X_train,y_train,X_test,y_test)

----